In [3]:
%pip install -q kagglehub





[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import os, shutil, stat, pathlib

src = "/Users/cynthiamcginnis/Downloads/kaggle (2).json"   # your file
dst_dir = pathlib.Path.home()/".kaggle"
dst = dst_dir/"kaggle.json"

dst_dir.mkdir(exist_ok=True)
shutil.copy2(src, dst)

# chmod 600
dst.chmod(stat.S_IRUSR | stat.S_IWUSR)

print("Saved credentials to:", dst)


Saved credentials to: /Users/cynthiamcginnis/.kaggle/kaggle.json


In [7]:
import subprocess, sys
# install kaggle if needed
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "kaggle"])

# simple sanity check: this should print your Kaggle account info JSON
subprocess.check_call(["kaggle", "config", "view"])



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


Configuration values from /Users/cynthiamcginnis/.kaggle
- username: cyleemcginnis
- path: None
- proxy: None
- competition: None


0

# Part II — MovieLens 1M with sklearn NMF (RMSE)

**Goal.** Load the MovieLens-1M ratings, predict missing ratings using **matrix factorization (NMF)** from `sklearn`, and report **RMSE** on a proper **train/test split**.  
We’ll also compute simple **baselines** for comparison (global mean, user mean, item mean).

**Key constraint.** `sklearn.decomposition.NMF` cannot natively handle missing values or observation masks; it minimizes error over *all* entries (including the many zeros we use to indicate "missing"). This is a core **limitation** we’ll discuss after reporting metrics.


In [12]:
# Step A — Download and unpack the dataset via Kaggle CLI 
import os, zipfile, glob, subprocess, sys, pandas as pd, numpy as np

DATA_DIR = "data_movielens1m"
os.makedirs(DATA_DIR, exist_ok=True)

# install kaggle if needed (no-op if already present)
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "kaggle"])

# download (idempotent: if the zip already exists, Kaggle keeps it)
subprocess.check_call([
    "kaggle", "datasets", "download",
    "-d", "odedgolden/movielens-1m-dataset",
    "-p", DATA_DIR
])

# unzip all zips in DATA_DIR
for zf in glob.glob(os.path.join(DATA_DIR, "*.zip")):
    with zipfile.ZipFile(zf, "r") as z:
        z.extractall(DATA_DIR)
    os.remove(zf)

print("Files in data folder:\n", "\n".join(sorted(os.listdir(DATA_DIR))))



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


Dataset URL: https://www.kaggle.com/datasets/odedgolden/movielens-1m-dataset
License(s): unknown

Files in data folder:
 README
movies.dat
ratings.dat
users.dat


100%|██████████| 5.83M/5.83M [00:00<00:00, 2.28GB/s]


## Step B — Load ratings 

Standardize the columns to: **userId, movieId, rating, timestamp**.


In [15]:
import pandas as pd, os, glob

def load_movielens_ratings(data_dir):
    # try .csv first
    csvs = glob.glob(os.path.join(data_dir, "*ratings*.csv"))
    if csvs:
        df = pd.read_csv(csvs[0])
        # normalize column names
        cols = {c.lower(): c for c in df.columns}
        # common names: userId, movieId, rating, timestamp
        return df.rename(columns={cols.get('userid','userId'): 'userId',
                                  cols.get('movieid','movieId'): 'movieId',
                                  cols.get('rating','rating'): 'rating',
                                  cols.get('timestamp','timestamp'): 'timestamp'})[['userId','movieId','rating','timestamp']]
    # try .dat (MovieLens 1M uses '::')
    dats = glob.glob(os.path.join(data_dir, "*ratings*.dat"))
    if dats:
        df = pd.read_csv(dats[0], sep="::", engine="python",
                         names=["userId","movieId","rating","timestamp"])
        return df
    raise FileNotFoundError("Could not find ratings file (.csv or .dat).")

ratings = load_movielens_ratings(DATA_DIR)
print(ratings.head())
print(ratings.shape, ratings['userId'].nunique(), ratings['movieId'].nunique())


   userId  movieId  rating  timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291
(1000209, 4) 6040 3706


## Step C — Train/Test split **by user** (prevent leakage)

We do a per-user split: for each user, hold out 20% of their ratings for **test** and keep 80% for **train**.  
This mirrors “predict unseen ratings for users we already know,” which is standard for CF.


In [19]:
from sklearn.model_selection import train_test_split

# sort for reproducibility
ratings = ratings.sort_values(['userId','timestamp']).reset_index(drop=True)

# build per-user train/test split indices
train_idx = []
test_idx  = []
rng = np.random.RandomState(42)

for uid, grp in ratings.groupby('userId', sort=False):
    idx = grp.index.to_numpy()
    if len(idx) <= 5:
        # tiny history users: keep all in train
        train_idx.extend(idx.tolist())
        continue
    # stratified by nothing here; just a random sample per user
    tr, te = train_test_split(idx, test_size=0.2, random_state=42)
    train_idx.extend(tr.tolist())
    test_idx.extend(te.tolist())

train_mask = np.zeros(len(ratings), dtype=bool); train_mask[train_idx] = True
test_mask  = np.zeros(len(ratings), dtype=bool); test_mask[test_idx]  = True

train = ratings[train_mask].copy()
test  = ratings[test_mask].copy()

print("Train/test sizes:", train.shape, test.shape)


Train/test sizes: (797758, 4) (202451, 4)


## Step D — Build user–item matrices and fit **sklearn NMF**

- Construct a sparse **R_train** matrix (users × items) with observed train ratings; zeros elsewhere.
- Fit `NMF(n_components = k)` on **R_train** (note: NMF “sees” zeros as real zeros, which is a limitation).
- Reconstruct `R_hat = W @ H` and evaluate **RMSE** on the **test** user-item pairs only.


In [24]:
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error

# map ids to consecutive indices
user_index = {u:i for i,u in enumerate(ratings['userId'].unique())}
item_index = {m:i for i,m in enumerate(ratings['movieId'].unique())}

n_users = len(user_index)
n_items = len(item_index)

def to_csr(df):
    ui = df['userId'].map(user_index).to_numpy()
    ii = df['movieId'].map(item_index).to_numpy()
    vv = df['rating'].astype(float).to_numpy()
    return csr_matrix((vv, (ui, ii)), shape=(n_users, n_items))

R_train = to_csr(train)

# --- Hyperparameter: number of latent factors (topics) ---
k = 20  # start with 20; you can try [10, 20, 40, 80]

nmf = NMF(
    n_components=k,
    init="nndsvda",
    max_iter=300,
    random_state=42,
    alpha_W=0.0, alpha_H=0.0, l1_ratio=0.0
)

W = nmf.fit_transform(R_train)   # (n_users × k)
H = nmf.components_              # (k × n_items)
R_hat = W @ H                    # dense reconstruction

# clip to rating scale
R_hat = np.clip(R_hat, 1.0, 5.0)

# Build arrays of predictions on the **test** pairs
u_te = test['userId'].map(user_index).to_numpy()
i_te = test['movieId'].map(item_index).to_numpy()
y_te = test['rating'].to_numpy().astype(float)
y_hat = R_hat[u_te, i_te]

rmse_nmf = mean_squared_error(y_te, y_hat, squared=False)
print(f"sklearn NMF (k={k}) — Test RMSE: {rmse_nmf:.4f}")


sklearn NMF (k=20) — Test RMSE: 2.4554


/Users/cynthiamcginnis/anaconda3/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 300 reached. Increase it to improve convergence.
  warnings.warn(


## Step E — Simple baselines 

- **Global mean** (predict everyone’s rating as the overall average).
- **User mean** (each user’s average).
- **Item mean** (each movie’s average).

These are strong sanity checks; if NMF can’t beat them, that’s a red flag.


In [27]:
global_mean = train['rating'].mean()

# user means backed off to global mean if user unseen (shouldn't happen here)
user_means = train.groupby('userId')['rating'].mean()
item_means = train.groupby('movieId')['rating'].mean()

y_hat_global = np.full_like(y_te, global_mean, dtype=float)
y_hat_user   = test['userId'].map(user_means).fillna(global_mean).to_numpy()
y_hat_item   = test['movieId'].map(item_means).fillna(global_mean).to_numpy()

rmse_global = mean_squared_error(y_te, y_hat_global, squared=False)
rmse_user   = mean_squared_error(y_te, y_hat_user,   squared=False)
rmse_item   = mean_squared_error(y_te, y_hat_item,   squared=False)

print(f"Global mean — Test RMSE: {rmse_global:.4f}")
print(f"User mean   — Test RMSE: {rmse_user:.4f}")
print(f"Item mean   — Test RMSE: {rmse_item:.4f}")


Global mean — Test RMSE: 1.1167
User mean   — Test RMSE: 1.0370
Item mean   — Test RMSE: 0.9796


## Step F — quick sweep over `k`

Try a few `n_components` values to see sensitivity of sklearn NMF.


In [30]:
ks = [10, 20, 40, 80]
rows = []
for k_ in ks:
    nmf_ = NMF(n_components=k_, init="nndsvda", max_iter=300, random_state=42)
    W_ = nmf_.fit_transform(R_train)
    H_ = nmf_.components_
    R_ = np.clip(W_ @ H_, 1.0, 5.0)
    y_hat_ = R_[u_te, i_te]
    rmse_ = mean_squared_error(y_te, y_hat_, squared=False)
    rows.append({"k": k_, "RMSE": rmse_})

rmse_table = pd.DataFrame(rows).sort_values("RMSE")
rmse_table


/Users/cynthiamcginnis/anaconda3/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 300 reached. Increase it to improve convergence.
  warnings.warn(
/Users/cynthiamcginnis/anaconda3/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 300 reached. Increase it to improve convergence.
  warnings.warn(


,k,RMSE
1,20,2.455406
2,40,2.465474
0,10,2.503531
3,80,2.506790


## Step G — Findings: **Limitations of sklearn NMF** (why RMSE may underperform)

- **No missing-value mask.** `sklearn`’s NMF minimizes reconstruction error over the *entire* matrix (including zeros we used as “missing”). In CF, zeros aren’t true ratings; this biases the factorization toward predicting many small values and can **hurt RMSE**.
- **No per-entry weighting.** We can’t tell NMF to “only care about observed ratings.” (There’s no `sample_weight`/mask in `sklearn` NMF.)
- **Rating scale vs. real preferences.** MovieLens ratings are **implicit+explicit** in practice; NMF with squared error on dense (zero-filled) matrices doesn’t match typical CF loss designs (e.g., only on observed entries, or with confidence weights).
- **Cold starts & bias terms.** Standard CF benefits from **user/item biases** and sometimes **regularized ALS** over observed entries. Plain NMF here lacks those inductive biases.

**Bottom line.** With plain `sklearn` NMF, it’s common to see **RMSE at or above simple baselines** (user/item means). That’s expected due to the constraints above.  scikit learn. (2009). 3.1. Cross-validation: Evaluating Estimator Performance — scikit-learn 0.21.3 Documentation. Scikit-Learn.org. https://scikit-learn.org/stable/modules/cross_validation.html




## Discussion: Why plain `sklearn` NMF performed poorly vs. simple baselines

**Results (this run)**  
- `sklearn` NMF (k=20): **RMSE ≈ 2.4554**  
- Global mean: **RMSE ≈ 1.1167**  
- User mean: **RMSE ≈ 1.0370**  
- Item mean: **RMSE ≈ 0.9796**  
- NMF sweep: k ∈ {10, 20, 40, 80} → RMSEs ≈ {2.50, 2.46, 2.47, 2.51} (all poor)

### Why this happens (core reasons)

1. **Wrong objective for matrix completion.**  
   `sklearn.decomposition.NMF` minimizes reconstruction error over **every matrix entry**. In sparse ratings, if you fill missings with zeros, NMF also tries to reconstruct those zeros. Because most entries are “missing→0,” the model focuses on explaining zeros instead of **observed** ratings, hurting RMSE on test pairs.

2. **No masking of missing entries.**  
   True matrix completion minimizes error **only on observed entries**. `sklearn` NMF cannot take a mask of observed indices, so it optimizes the wrong loss when missings are zero-filled.

3. **No bias terms (μ, user bias, item bias).**  
   Strong baselines use  
   $
   \hat r_{ui} = \mu + b_u + b_i
   \$ 
   capturing user generosity and item popularity. Plain NMF has **no biases**, so factors must approximate what simple biases explain easily—degrading accuracy.

4. **Non-negativity + lack of centering.**  
   Recommender models benefit from centering to residuals (above/below zero). NMF requires **non-negative** inputs and factors, preventing proper centering and making it ill-suited for rating residuals.

5. **Tuning can’t fix the mismatch.**  
   Changing `k` (10→80) doesn’t help because the fundamental **objective mismatch** dominates.

---

### How to fix / improve

**A. Use matrix-completion algorithms with masking + biases**
- **SVD/SVD++ with biases** (e.g., Surprise’s `SVD`):  
  $
  \hat r_{ui} = \mu + b_u + b_i + p_u^\top q_i
  \$ 
  Optimizes **only on observed entries** with regularization.
- **ALS/SGD with an observed-entry mask** (e.g., custom explicit-feedback ALS).

**B. Add strong baselines**
- **User-/item-mean** (done), plus **kNN (user/item)** on **co-rated** sets (cosine/Pearson). Item-kNN is often very competitive on MovieLens-like data.

**C. If you must stay within `sklearn`**
- Fit a **bias model** first, then factorize **residuals** with a method that allows negatives (e.g., `TruncatedSVD` on centered residuals), not NMF.
- Or use a third-party **masked NMF** implementation.

**D. Practical recipe (drop-in path to better RMSE)**

1. Estimate global and bias terms on the training set: $\mu,\; b_u,\; b_i$.
2. Form residuals for **observed** ratings: $r_{ui} - \mu - b_u - b_i$.
3. Fit a factor model (SVD/ALS) on **observed residuals only** with L2 regularization.
4. Predict with the bias-aware factorization:
   $
   \hat r_{ui} \;=\; \mu \;+\; b_u \;+\; b_i \;+\; \mathbf{p}_u^\top \mathbf{q}_i.
   \$.
5. Evaluate test error with RMSE:
   $
   \mathrm{RMSE} \;=\; \sqrt{\frac{1}{|\Omega_{\text{test}}|}\sum_{(u,i)\in\Omega_{\text{test}}}\big(r_{ui}-\hat r_{ui}\big)^2 }.
   \$.


---

### Bottom line
Plain `sklearn` NMF is solving the **wrong objective** for recommender data, so it loses to even trivial baselines. Move to a **masked, bias-aware** matrix-completion method (e.g., Surprise SVD / ALS) or use **similarity baselines** (item-kNN) to reach—and typically beat—the item-mean RMSE.
